In [68]:
import osmnx as ox
from shapely.geometry import Polygon, MultiPolygon
import geopandas as gpd
import requests 
from collections import defaultdict
from ipynb.fs.full.transportationNetworks import bounding_coordinates, flatten_coords
import csv

In [71]:
def count_objects_in_polygon(polygon):
    if not isinstance(polygon, (Polygon, MultiPolygon)):
        raise TypeError("Not polygon obj")

    shop_types = ['grocery', 'general', 'supermarket', 'deli', 'greengrocer', 'food', 'marketplace', 'kiosk']
    

    grocery_stores = ox.features.features_from_polygon(polygon, tags={"shop":shop_types, "amenity":shop_types})
    
    shop_counts = defaultdict(int)
    
    # Iterate over each grocery store and increment the count for its shop type
    shop_counts = defaultdict(int)
    for shop_type in shop_types:
        num_shops = len(grocery_stores[(grocery_stores['shop'] == shop_type) | (grocery_stores['amenity'] == shop_type)])
        shop_counts[shop_type] = num_shops
    
    return dict(shop_counts)

In [79]:
def return_weight(city, country):
    poly_coords = bounding_coordinates(city, country)[0]
    poly_coords = flatten_coords(poly_coords)
    poly = Polygon(poly_coords)


    object_counts = count_objects_in_polygon(poly)
    square_surface_area = poly.area
    
    weightSum = object_counts['grocery']*3 + object_counts['general']*3 + object_counts['supermarket']*3 + object_counts['deli'] *2 + object_counts['greengrocer'] * 2 + object_counts['food'] * 1 + object_counts['marketplace']*3 + object_counts['kiosk']*1

    print(weightSum, square_surface_area)
    return weightSum/square_surface_area


In [81]:
def test():
    print(return_weight('Cidade De Chimoio', 'Mozambique'))


In [ ]:

def update_grocery_density(csv_filepath):
    # Read the CSV data
    with open(csv_filepath, newline='') as file:
        reader = csv.DictReader(file)
        rows = list(reader)  # Convert iterator to list to reuse it

    for row in rows:
        print("Checking:", row['Region'].replace("_", " "), row['Country'].replace("_", " "))
        result = return_weight(row['Region'].replace("_", " "), row['Country'].replace("_", " "))
        print(row['Region'], row['Country'], result)
        row['Food_Storage_Groceries'] = float(row['Food_Storage_Groceries']) + result

    with open(csv_filepath, 'w', newline='') as file:
        fieldnames = rows[0].keys()  # Grab the field names from the first row
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)